In [94]:
import requests
import pandas as pd
import datetime
"""
For testing on local machine

The function works by requesting a json of the latest road incidents from the LTA Dynamic Data API. This lambda is queried hourly. After quering the data, 
it checks if the api returns any data. If there is, it converts it into a pandas table, and creates the Date and Time column. It then filters out to keep the data
from the past hourly, and checks again if there is any data. If there is, it converts it into a json table, and is inserted into the DynamoDB table.
"""

# URL and Pulling Data
traffic_url = "http://datamall2.mytransport.sg/ltaodataservice/TrafficIncidents"
api_key = open("api_key.txt").read()
api_headers = {"AccountKey": api_key, "accept": "application/json"}

request = requests.get(url=traffic_url, headers=api_headers)
print(request)
data = request.json()
dt = datetime.datetime.now()    # + datetime.timedelta(hours = 8)
                                # Only needed in AWS due to time difference

df = pd.DataFrame(data["value"])

<Response [200]>


In [95]:
df = pd.DataFrame(data["value"])
df

,Type,Latitude,Longitude,Message
0,Roadwork,1.311912,103.803815,(24/1)21:56 Roadworks on Farrer Road (towards ...
1,Vehicle breakdown,1.412663,103.806420,(24/1)21:55 Vehicle breakdown on SLE (towards ...
2,Obstacle,1.398875,103.898714,(24/1)20:43 Obstacle on TPE (towards SLE) at P...


In [14]:
len(df) # Add line to break

if (len(df) > 0):
    print()
    # Continue with code

In [96]:
# Extract Date and Time out of the Message to keep the main message only

date_regex = "([0-9]{1,2}/[0-9]{1,2})"
df["Date"] = df["Message"].str.extract(pat = date_regex) + "/" + str(dt.year)

time_regex = "([0-9]{1,2}:[0-9]{1,2})"
df["Time"] = df["Message"].str.extract(pat = time_regex)

df["Date_Time_Start"] = pd.to_datetime(df["Date"] + df["Time"], format = "%d/%m/%Y%H:%S")

message_regex = "\d\s(.*$)"
df["Message"] = df["Message"].str.extract(pat = message_regex)

df["Date_Time_End"] = "Nil"

df = df[["Type", "Date_Time_Start", "Date_Time_End", "Message", "Latitude", "Longitude"]]

In [98]:
df["Date_Time_Start"].astype(str)

0    2023-01-24 21:00:56
1    2023-01-24 21:00:55
2    2023-01-24 20:00:43
Name: Date_Time_Start, dtype: object

In [116]:
# Filtering to keep only previous hour of data (Removed)
if dt.hour >= 1:
    prev_hour = str(dt.hour - 1)
    df_boolean = df["Time"].str.match(pat = (prev_hour + ":"))
else:
    prev_hour = "23"
    df_boolean = df["Time"].str.match(pat = "23:")

KeyError: 'Time'

In [117]:
# Filtering to keep only past 15 minutes of data
dt_last15 = dt-datetime.timedelta(minutes=15)
print(dt_last15)

2023-01-08 19:00:41.755815


In [118]:
df_tocloud = df[df['Date_Time'] > dt_last15].reset_index(drop = True)
df_tocloud

,Type,Date_Time,Message,Latitude,Longitude


In [119]:
df_tocloud_key = dt.strftime(format = "%Y-%m-%d %H:%m")
df_tocloud_key

'2023-01-08 19:01'

In [128]:
dt = datetime.datetime.now()
dt.strftime(format = "%Y-%m-%d %H:%M")

'2023-01-09 22:44'

## For Building Temp and End Time

In [83]:
current = df[4:9].reset_index(drop = True)
temp = df.reset_index(drop = True)

In [84]:
current["key"] = current["Type"] + " " + current["Message"]
temp["key"] = temp["Type"] + " " + temp["Message"]

In [85]:
check_active = temp["key"].isin(current["key"])
check_active

0    False
1    False
2    False
3    False
4     True
5     True
6     True
7     True
8     True
Name: key, dtype: bool

In [86]:
temp = temp[~check_active] #Inverse to keep only those that are expired, i.e. incident is resolved and is not active
temp["Date_Time_End"] = dt.strftime(format = "%Y-%m-%d %H:%M:%S")
temp = temp[["Type", "Date_Time_Start", "Date_Time_End", "Message", "Latitude", "Longitude"]]
temp

,Type,Date_Time_Start,Date_Time_End,Message,Latitude,Longitude
0,Accident,2023-01-22 18:00:11,2023-01-22 18:14:36,Accident on PIE (towards Changi Airport) after...,1.327986,103.824782
1,Vehicle breakdown,2023-01-22 17:00:59,2023-01-22 18:14:36,Vehicle breakdown on KJE (towards BKE) after C...,1.383928,103.738290
2,Vehicle breakdown,2023-01-22 17:00:51,2023-01-22 18:14:36,Vehicle breakdown on SLE (towards BKE) after U...,1.405386,103.809720
3,Accident,2023-01-22 16:00:37,2023-01-22 18:14:36,Accident on PIE (towards Tuas) after Stevens R...,1.327868,103.824831


,Type,Date_Time_Start,Date_Time_End,Message,Latitude,Longitude,key
4,Heavy Traffic,2023-01-22 14:00:08,Nil,Heavy Traffic on CTE (towards AYE) at PIE(Chan...,1.344612,103.860143,Heavy Traffic Heavy Traffic on CTE (towards AY...


## For Testing and Subsequent Wring

In [92]:
datetime.(1674596640000)

SyntaxError: invalid syntax (2509452883.py, line 1)

In [120]:
# Converting to JSON for Writing to Cloud
df_tocloud = df_tocloud.to_json()


# Reading from Cloud
import json
df_raw = json.loads(df_tocloud)
df_local = pd.DataFrame(df_raw)
df_local

,Type,Date_Time,Message,Latitude,Longitude


,Type,Latitude,Longitude,Message
0,Vehicle breakdown,1.295601,103.892536,(24/1)22:13 Vehicle breakdown on ECP (towards ...
1,Roadwork,1.311912,103.803815,(24/1)21:56 Roadworks on Farrer Road (towards ...
2,Vehicle breakdown,1.412663,103.806420,(24/1)21:55 Vehicle breakdown on SLE (towards ...
3,Roadwork,1.348464,103.792167,(24/1)21:31 Roadworks on BKE (towards PIE) bef...
4,Obstacle,1.398875,103.898714,(24/1)20:43 Obstacle on TPE (towards SLE) at P...
